## Import

In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from datetime import datetime, timedelta
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_curve, roc_auc_score
from sklearn.model_selection import StratifiedKFold
from itertools import chain
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
import xgboost as xgb

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense,Dropout,BatchNormalization,Activation
from keras.models import load_model

from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

Morandi = ['#2D241F','#614E52','#7A6747','#A06A50','#ADAEB2','#69647B',
           '#E2CDBC','#A56C41','#B17A7D','#A68782',
          '#903B1C','#92ACD1','#D0C1C6','#D6D6D6','#511F1E']

major_apt = ['ATL','BOS','BWI','CLT','DCA','DEN','DFW','DTW','EWR','FLL','HNL','IAD',
             'IAH','JFK','LAS','LAX','LGA','MCO','MDW','MEM','MIA','MSP','ORD','PHL',
             'PHX','PDX','SAN','SEA','SFO','SLC','TPA','LGB','OAK','SNA']

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

import warnings
warnings.simplefilter('ignore')

Using TensorFlow backend.


In [2]:
datetime_cols = ['CRS_DEP_DT', 'DEP_DELAY_TD', 'CRS_ELAPSED_TD', 'DEP_DT',
                'CRS_ARR_DT','ARR_DELAY_TD', 'ARR_DT']
testing = pd.read_csv('testing.csv',index_col=0,parse_dates=datetime_cols)

/opt/anaconda/envs/Python3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
testing.head()

,FL_DATE,OP_UNIQUE_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN,DEST_AIRPORT_ID,DEST,DEP_DELAY,ARR_DELAY,...,pressure,tempC,uvIndex,visibility,winddir16Point,winddirDegree,windspeedKmph,weatherDesc_new,Wide,Year
43767,2018-09-24,WN,N439WN,4597,11618,EWR,11292,DEN,-3.0,-15.0,...,1031,19,5,10,E,79,23,Sunny,0.0,2003.0
43768,2018-09-24,UA,N18119,2216,11618,EWR,11292,DEN,-7.0,-5.0,...,1031,19,5,10,E,79,23,Sunny,0.0,1997.0
43769,2018-09-24,UA,N38467,2415,11618,EWR,14057,PDX,-4.0,-13.0,...,1031,19,5,10,E,79,23,Sunny,0.0,2013.0
43770,2018-09-24,DL,N944AT,2027,11618,EWR,10397,ATL,-8.0,-13.0,...,1031,19,5,10,E,79,23,Sunny,0.0,1999.0
43771,2018-09-24,UA,N215UA,1148,11618,EWR,12892,LAX,1.0,1.0,...,1031,19,5,10,E,79,23,Sunny,1.0,2000.0


In [4]:
testing.shape

(1765464, 43)

## Feature Engineering

### Find previous flights

In [5]:
trn_mg_all = pd.DataFrame(columns=['TAIL_NUM', 'DEP_DT', 'ORIGIN', 'DEST', 'ARR_DT'])

for i in range(1,53,13):
#    trn_mg_ori = training[(training['Week_no']>=i)&(training['Week_no']<i+13)&
#                          (training['ORIGIN'].isin(major_apt))][['TAIL_NUM','ORIGIN','DEP_DT']]
    trn_mg_ori = testing[(testing['Week_no']>=i)&(testing['Week_no']<i+13)][['TAIL_NUM','ORIGIN','DEP_DT']]
#     trn_mg_dest = training[(training['Week_no']>=i)&(training['Week_no']<i+13)&
#                           (training['DEST'].isin(major_apt))][['TAIL_NUM','DEST','ARR_DT']]
    trn_mg_dest = testing[(testing['Week_no']>=i)&(testing['Week_no']<i+13)][['TAIL_NUM','DEST','ARR_DT']]
    
    trn_mg = pd.merge(trn_mg_ori,trn_mg_dest,
                      left_on=['TAIL_NUM','ORIGIN'],
                      right_on=['TAIL_NUM','DEST'],how='inner')
    
    trn_mg_first = trn_mg[trn_mg['DEP_DT']>
                          trn_mg['ARR_DT']].sort_values(by=['TAIL_NUM','DEP_DT','ARR_DT'],
                                                        ascending=[True,True,False])
    
    trn_mg_first = trn_mg_first.groupby(['TAIL_NUM','DEP_DT']).first().reset_index()
    
    trn_mg_all = trn_mg_all.append(trn_mg_first)
    
    print(i)

1
14
27
40


In [6]:
trn_mg_all.reset_index(drop=True,inplace=True)

In [7]:
trn_mg_all['Turnaround_time'] = trn_mg_all['DEP_DT'] - trn_mg_all['ARR_DT']

In [8]:
trn_mg_all.head()

,TAIL_NUM,DEP_DT,ORIGIN,DEST,ARR_DT,Turnaround_time
0,216NV,2018-09-27 11:49:00,EWR,EWR,2018-09-27 10:40:00,01:09:00
1,219NV,2018-09-30 11:49:00,EWR,EWR,2018-09-30 10:29:00,01:20:00
2,221NV,2018-09-30 10:12:00,LAX,LAX,2018-09-30 08:55:00,01:17:00
3,221NV,2018-09-30 18:22:00,CVG,CVG,2018-09-30 17:15:00,01:07:00
4,221NV,2018-09-30 20:26:00,DEN,DEN,2018-09-30 19:08:00,01:18:00


In [9]:
testing_prev = pd.merge(testing,trn_mg_all,
                         left_on=['TAIL_NUM','ORIGIN','DEP_DT'],
                         right_on=['TAIL_NUM','ORIGIN','DEP_DT'],
                         how='inner',
                         suffixes=('','_prev'))

In [10]:
testing_prev = pd.merge(testing_prev,
                         testing_prev[['TAIL_NUM','DEST','ARR_DT','CRS_ARR_DT','DISTANCE','DEP_DELAY']],
                         left_on=['TAIL_NUM','ORIGIN','ARR_DT_prev'],
                         right_on=['TAIL_NUM','DEST','ARR_DT'],
                         how='inner',
                         suffixes=('','_prev'))

### Create dependent variable

In [11]:
def binarise(data):
    if data <= 15:
        return 0
    else:
        return 1
    
testing_prev['DEP_DELAY_BI'] = testing_prev['DEP_DELAY'].apply(binarise)

### Airport average delay in the last 6 hours

In [15]:
testing_prev['Last2'] = testing_prev['CRS_DEP_DT'] - timedelta(seconds=60*60*2)
testing_prev['Last8'] = testing_prev['CRS_DEP_DT'] - timedelta(seconds=60*60*8)
testing_prev['Last14'] = testing_prev['CRS_DEP_DT'] - timedelta(seconds=60*60*14)

In [16]:
apr_avg_delay = pd.DataFrame(columns=['ORIGIN', 'DEP_DT', 'OP_CARRIER_FL_NUM', 
                                      'OP_UNIQUE_CARRIER','AVG_DELAY6'])
for i in range(1,53,1):
    pre_mg1 = testing_prev[(testing_prev['Week_no']>=i)&
                           (testing_prev['Week_no']<i+1)][['ORIGIN','Last2','Last8','DEP_DT',
                                                            'OP_CARRIER_FL_NUM',
                                                            'OP_UNIQUE_CARRIER']]
    
    pre_mg2 = testing_prev[(testing_prev['Week_no']>=i-1)&
                           (testing_prev['Week_no']<i+1)][['ORIGIN','DEP_DT','DEP_DELAY']]
    
    avg_delay = pysqldf("""SELECT a.ORIGIN,
                                  a.DEP_DT,
                                  a.OP_CARRIER_FL_NUM,
                                  a.OP_UNIQUE_CARRIER,
                                  avg(a.DEP_DELAY) as AVG_DELAY6
                           FROM 
                              (SELECT *
                               FROM pre_mg1 a 
                                    INNER JOIN pre_mg2 b 
                                    on a.ORIGIN = b.ORIGIN
                                       and a.Last2 >= b.DEP_DT
                                       and a.Last8 <= b.DEP_DT) a
                            GROUP BY a.ORIGIN,a.DEP_DT,
                                     a.OP_CARRIER_FL_NUM,a.OP_UNIQUE_CARRIER
                            ;""")

    apr_avg_delay = apr_avg_delay.append(avg_delay)
    
    print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52


In [17]:
# Merge back to the training_prev 
apr_avg_delay['DEP_DT'] = pd.to_datetime(apr_avg_delay['DEP_DT'])
testing_prev = pd.merge(testing_prev,apr_avg_delay,
                         left_on=['ORIGIN', 'DEP_DT', 
                                  'OP_CARRIER_FL_NUM','OP_UNIQUE_CARRIER'],
                         right_on=['ORIGIN', 'DEP_DT',
                                  'OP_CARRIER_FL_NUM','OP_UNIQUE_CARRIER'],
                         how='left',
                         suffixes=('','_day'))

In [18]:
#  rows don't have avg_delay for the last 6 hours due to no flights in the airport 
testing_prev[testing_prev['AVG_DELAY6'].isnull()].shape

(110590, 56)

In [19]:
# fill the missing value with 0 since if it is missing, it normally means there is no flights
testing_prev['AVG_DELAY6'] = testing_prev['AVG_DELAY6'].fillna(0)

In [20]:
sum(testing_prev['AVG_DELAY6'].isnull())

0

### Estimated turnaround time

In [22]:
testing_prev['EST_TRN'] = testing_prev['CRS_DEP_DT'] - testing_prev['CRS_ARR_DT_prev']

### OD pair delay status

In [23]:
def get_od_name(x):
    if x['ORIGIN'] > x['DEST']:
        return x['DEST'] + '-' + x['ORIGIN']
    elif x['ORIGIN'] < x['DEST']:
        return x['ORIGIN'] + '-' + x['DEST']

testing_prev['OD'] = testing_prev[['ORIGIN','DEST']].apply(get_od_name,axis=1)

In [24]:
od_delay_total = pd.DataFrame(columns=['OD', 'ORIGIN', 'DEP_DT', 'OP_CARRIER_FL_NUM',
                                       'OP_UNIQUE_CARRIER','OD_DELAY6'])
for i in range(1,53,4):
    pre_mg1 = testing_prev[(testing_prev['Week_no']>=i)&
                           (testing_prev['Week_no']<i+4)][['OD','ORIGIN','Last2','Last8','DEP_DT',
                                                            'OP_CARRIER_FL_NUM',
                                                            'OP_UNIQUE_CARRIER']]
    
    pre_mg2 = testing_prev[(testing_prev['Week_no']>=i-1)&
                           (testing_prev['Week_no']<i+4)][['OD','ORIGIN','DEP_DT','DEP_DELAY']]
    
    od_delay = pysqldf("""SELECT a.OD,
                                  a.ORIGIN,
                                  a.DEP_DT,
                                  a.OP_CARRIER_FL_NUM,
                                  a.OP_UNIQUE_CARRIER,
                                  avg(a.DEP_DELAY) as OD_DELAY6
                           FROM 
                              (SELECT a.*,
                                      b.DEP_DELAY
                               FROM pre_mg1 a 
                                    INNER JOIN pre_mg2 b 
                                    on a.OD = b.OD
                                       and a.Last2 >= b.DEP_DT
                                       and a.Last8 <= b.DEP_DT) a
                           GROUP BY a.OD,a.ORIGIN,a.DEP_DT,
                                    a.OP_CARRIER_FL_NUM,a.OP_UNIQUE_CARRIER
                            ;""")

    od_delay_total = od_delay_total.append(od_delay)

In [25]:
od_delay_total['DEP_DT'] = pd.to_datetime(od_delay_total['DEP_DT'])
testing_prev = pd.merge(testing_prev,od_delay_total,
                         left_on=['OD','ORIGIN', 'DEP_DT', 
                                  'OP_CARRIER_FL_NUM','OP_UNIQUE_CARRIER'],
                         right_on=['OD','ORIGIN', 'DEP_DT',
                                  'OP_CARRIER_FL_NUM','OP_UNIQUE_CARRIER'],
                         how='left')

In [26]:
print('the number of missing values: ' + str(sum(testing_prev['OD_DELAY6'].isnull())))

the number of missing values: 313970


In [27]:
testing_prev['OD_DELAY6'] = testing_prev['OD_DELAY6'].fillna(0)

In [28]:
sum(testing_prev['OD_DELAY6'].isnull())

0

### Minimum Turnaround Time

In [29]:
min5_trn = testing_prev.groupby(['OP_UNIQUE_CARRIER','OP_CARRIER_FL_NUM','ORIGIN','DEST'])['Turnaround_time'].quantile(.05)
min5_trn = pd.DataFrame(min5_trn).reset_index()
min5_trn['Turnaround_time'] = min5_trn['Turnaround_time'].dt.seconds/60
min5_trn.rename({'Turnaround_time':'MIN5_TRN'},axis=1,inplace=True)

In [30]:
testing_prev = pd.merge(testing_prev,min5_trn,
                         left_on=['OP_UNIQUE_CARRIER','OP_CARRIER_FL_NUM','ORIGIN','DEST'],
                         right_on=['OP_UNIQUE_CARRIER','OP_CARRIER_FL_NUM','ORIGIN','DEST'],
                         how = 'left')

### Turnaround buffer time (EST_TRN - MIN_TRN)

In [31]:
testing_prev['Buff_TRN'] = (testing_prev['EST_TRN'].dt.seconds/60 
                            - testing_prev['MIN5_TRN']).apply(lambda x: 0 if x <= 0 else x)

### Export testing_prev

In [35]:
testing_prev.to_csv('./test/testing.csv')

## Prepare for LSTM

### Preprocessing

In [36]:
Maj_ORI = testing_prev[testing_prev['ORIGIN'].isin(major_apt)]
CRS_DEP_DT = Maj_ORI['CRS_DEP_DT']
Maj_ORI = pd.DataFrame(Maj_ORI.groupby(['ORIGIN',
                                        CRS_DEP_DT.dt.date,
                                        CRS_DEP_DT.dt.hour]).agg({'DEP_DELAY':['count','mean']}))
Maj_ORI.index.names = ['ORIGIN','CRS_DEP_DT','CRS_DEP_Hr']
Maj_ORI = Maj_ORI.reset_index()
Maj_ORI.columns = Maj_ORI.columns.droplevel()
Maj_ORI.columns = ['ORIGIN','CRS_DEP_DT','CRS_DEP_Hr','DEP_CNT','AVG_DEP_DELAY']

In [37]:
Maj_DEST = testing_prev[testing_prev['DEST'].isin(major_apt)]
CRS_ARR_DT = Maj_DEST['CRS_ARR_DT']
Maj_DEST = pd.DataFrame(Maj_DEST.groupby(['DEST',
                                          CRS_ARR_DT.dt.date,
                                          CRS_ARR_DT.dt.hour]).size())
Maj_DEST.index.names = ['DEST','CRS_ARR_DT','CRS_ARR_Hr']
Maj_DEST = Maj_DEST.reset_index()
Maj_DEST.columns = ['DEST','CRS_ARR_DT','CRS_ARR_Hr','ARR_CNT']

In [44]:
from itertools import product
base = pd.date_range('2018-09-24', periods=2376, freq='1H')
base = list(product(base,major_apt))
base_df = pd.DataFrame(base,columns=['CRS_DEP_DT','ORIGIN'])
base_df['CRS_DEP_Hr'] = base_df['CRS_DEP_DT'].dt.hour
base_df['Weekday'] = base_df['CRS_DEP_DT'].dt.weekday
base_df['Month'] = base_df['CRS_DEP_DT'].dt.month
base_df['Day'] = base_df['CRS_DEP_DT'].dt.day
base_df['CRS_DEP_DT'] = base_df['CRS_DEP_DT'].dt.date

In [45]:
base_df = pd.merge(base_df,Maj_ORI,
                   left_on=['ORIGIN','CRS_DEP_DT','CRS_DEP_Hr'],
                   right_on=['ORIGIN','CRS_DEP_DT','CRS_DEP_Hr'],
                   how='left')

In [46]:
base_df = base_df.fillna(0)

In [47]:
base_df = pd.merge(base_df,Maj_DEST,
                   left_on=['ORIGIN','CRS_DEP_DT','CRS_DEP_Hr'],
                   right_on=['DEST','CRS_ARR_DT','CRS_ARR_Hr'],
                   how='left')

In [48]:
base_df = base_df.fillna(0)
base_df.drop(['DEST','CRS_ARR_DT','CRS_ARR_Hr'],axis=1,inplace=True)

### Merge with the weather data

In [49]:
all_weather = pd.read_csv('all_weather_data.csv')

In [50]:
all_weather['Datetime'] = pd.to_datetime(all_weather['Datetime']).dt.date

In [51]:
all_weather = all_weather[['Airport_name','DewPointC','cloudcover',
                           'precipMM','visibility','Datetime','Hour']]

In [52]:
base_wea = pd.merge(base_df,all_weather,
                    left_on=['ORIGIN','CRS_DEP_DT','CRS_DEP_Hr'],
                    right_on=['Airport_name','Datetime','Hour'],
                    how='left') 

In [53]:
base_wea.drop(['Airport_name','Datetime','Hour'],axis=1,inplace=True)

In [54]:
base_wea.shape

(80784, 13)

### Set average delay to 0 for those hours with only 1 or 2 flights

In [55]:
base_wea['AVG_DEP_DELAY'][base_wea['DEP_CNT'] <= 2] = 0

### Export LSTM testing

In [56]:
base_wea.to_csv('lstm_testing.csv')